### Import softwareQ packages

In [1]:
import pystaq # staq quantum compiler
import pyqpp  # Quantum++ simulation engine

### Parse some QASM code and expand broadcasted operations

In [2]:
circuit = pystaq.parse_file("GHZ.qasm")
print(circuit)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
h q[0];
cx q[0],q[1];
cx q[0],q[2];
measure q -> c;



### Expand broadcasted operations

In [3]:
pystaq.desugar(circuit)
print(circuit)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
h q[0];
cx q[0],q[1];
cx q[0],q[2];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];



### Create a test linear QPU with 3 qubits and write it to a JSON file

In [4]:
device = pystaq.Device(3)
device.add_edge(0, 1)
device.add_edge(1, 2)

with open("device.json", "w") as file:
    file.write(str(device))
    
print(device)

{
  "couplings": [
    {
      "control": 0,
      "target": 1
    },
    {
      "control": 1,
      "target": 0
    },
    {
      "control": 1,
      "target": 2
    },
    {
      "control": 2,
      "target": 1
    }
  ],
  "name": "Custom Device",
  "qubits": [
    {
      "id": 0
    },
    {
      "id": 1
    },
    {
      "id": 2
    }
  ]
}


### Map the circuit onto the QPU

In [5]:
pystaq.map(prog=circuit, layout="linear", mapper="swap", device_json_file="device.json")
print(circuit)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
U(pi/2,0,pi) q[0];
CX q[0],q[1];
CX q[0],q[1];
CX q[1],q[0];
CX q[0],q[1];
CX q[1],q[2];
measure q[1] -> c[0];
measure q[0] -> c[1];
measure q[2] -> c[2];



### Try again, with a better mapping algorithm

In [6]:
# Since mapping happens in-place, we must reparse and reprocess the circuit
circuit = pystaq.parse_file("GHZ.qasm")
pystaq.desugar(circuit)
pystaq.map(prog=circuit, layout="linear", mapper="steiner", device_json_file="device.json")
print(circuit)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[3];
creg c[3];
U(pi/2,0,pi) q[0];
CX q[1],q[2];
CX q[0],q[1];
CX q[1],q[2];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];



### Convert to a different API/QDK (e.g., Google's Cirq)

In [7]:
print(circuit.to_cirq())

import cirq
import numpy as np
from cmath import pi,exp,sin,cos,tan,log as ln,sqrt

class UGate(cirq.SingleQubitMatrixGate):
    def __init__(self, theta, phi, lambd):
        mat = np.matrix([[exp(-1j*(phi+lambd)/2)*cos(theta/2),
                          -exp(-1j*(phi-lambd)/2)*sin(theta/2)],
                         [exp(1j*(phi-lambd)/2)*sin(theta/2),
                          exp(1j*(phi+lambd)/2)*cos(theta/2)]])
        cirq.SingleQubitMatrixGate.__init__(self, mat)
        self.theta = theta
        self.phi = phi
        self.lambd = lambd

    def __str__(self):
        return str(self.__class__.__name__) + "(" + str(self.theta) + "," \
               + str(self.phi) + "," + str(self.lambd) + ")"

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.theta == other.theta \
                   & self.phi == other.phi \
                   & self.lambd == other.lambd
        else:
            return False

    def __ne__(self, other):
 

### Write the circuit to file and run 100 simulations on a noisy engine

In [8]:
with open("circuit.qasm", "w") as file:
    file.write(str(circuit))
    
qcircuit = pyqpp.qasm.read_from_file("circuit.qasm")

qengine = pyqpp.QNoisyEngine(qcircuit, pyqpp.QubitDepolarizingNoise(0.01))
result = qengine.execute(100)

print(result)

[QNoisyEngine]
<QCircuit nq: 3, nc: 3, d: 2>
last probs: [0.5, 1, 1]
last dits: [0, 0, 0]
stats:
	reps: 100
	outcomes: 5
	[0 0 0]: 50
	[0 0 1]: 2
	[0 1 0]: 4
	[1 1 0]: 1
	[1 1 1]: 43
